In [6]:
import pandas as pd
import numpy as np
import json
import math
import sys
import os
import json
sys.path.append("./levenshtein.py")  # path contains python_file.py
from levenshtein import levenshtein as levenshtein
sys.path.append("./candidates.py")  # path contains python_file.py
from candidates import candidates as candidates

In [2]:
candidates[::-1]

[{'nameAR': 'سيف الدين مخلوف', 'nameFR': 'Seifeddine Makhlouf'},
 {'nameAR': 'عبيد بريكي', 'nameFR': 'Abid Briki'},
 {'nameAR': 'حاتم بولبيار', 'nameFR': 'Hatem Boulabiar'},
 {'nameAR': 'الناجي جلول', 'nameFR': 'Néji Jalloul'},
 {'nameAR': 'أحمد الصافي سعيد', 'nameFR': 'Safi Saïd'},
 {'nameAR': 'سعيد العايدي', 'nameFR': 'Saïd Aïdi'},
 {'nameAR': 'سلمى اللومي', 'nameFR': 'Selma Elloumi Rekik'},
 {'nameAR': 'سليم الرياحي', 'nameFR': 'Slim Riahi'},
 {'nameAR': 'الياس الفخفاخ', 'nameFR': 'Elyes Fakhfakh'},
 {'nameAR': 'قيس سعيّد', 'nameFR': 'Kaïs Saïed'},
 {'nameAR': 'يوسف الشاهد', 'nameFR': 'Youssef Chahed'},
 {'nameAR': 'عمر منصور', 'nameFR': 'Omar Mansour'},
 {'nameAR': 'عبد الفتاح مورو', 'nameFR': 'Abdelfattah Mourou'},
 {'nameAR': 'محمد الهاشمي حامدي', 'nameFR': 'Hechmi Hamdi'},
 {'nameAR': 'محمد الصغير نوري', 'nameFR': 'Mohamed Esghaier Nouri'},
 {'nameAR': 'محسن مرزوق', 'nameFR': 'Mohsen Marzouk'},
 {'nameAR': 'عبد الكريم زبيدي', 'nameFR': 'Abdelkrim Zbidi'},
 {'nameAR': 'محمد المنص

In [7]:
files = os.listdir('./sheets/Ariana')

In [8]:
def candidateName(name):
    current_max_value = 0
    current_max_name = name
    for candidate in candidates:
        distance = levenshtein(name[::-1],candidate['nameAR'])
        if(distance>current_max_value):
            current_max_name,current_max_value = candidate['nameFR'],distance
    return current_max_name

In [9]:
result = []
for filename in files:
    print("\x1b[31mScrapping ",filename,"\x1b[0m")
    file = pd.ExcelFile("./sheets/Ariana/"+filename)
    nb_sheets = len(file.sheet_names)
    

    sheets = [file.parse(i).values for i in range(nb_sheets) if file.parse(i).values.size>0]
    sheets.reverse()
    fullMatrix = np.fliplr(np.hstack(tuple(sheets)))
    first_candidate_line = [i for i in range(len(fullMatrix[:,0])) if fullMatrix[i,0] == 1][0]#fullMatrix.shape[0] - 26
    print("--First line candidate",first_candidate_line)
    matrix_rows = first_candidate_line+26
    data  = dict([(candidateName(fullMatrix[i,1]),sum([j for j in fullMatrix[i,2:] if isinstance(j, int)])) for i in range(first_candidate_line,matrix_rows)])
    result.append({'delegationName':filename.replace('أريانة-','').replace('.xlsx',''),'data':data})
    
out = json.dumps(result,indent=4,ensure_ascii=False)
print(out)
outfile = open('output/Ariana.json',"w+",encoding="utf-8")
#outfile.write(out)

Scrapping  أريانة-أريانة-المدينة_2.xlsx 
--First line candidate 10
Scrapping  أريانة-المنيهلة_2.xlsx 
--First line candidate 10
Scrapping  أريانة-حي-التضامن.xlsx 
--First line candidate 10
Scrapping  أريانة-رواد.xlsx 
--First line candidate 10
Scrapping  أريانة-سكرة.xlsx 
--First line candidate 11
Scrapping  أريانة-سيدي-ثابت.xlsx 
--First line candidate 11
Scrapping  أريانة-قلعة-الأندلس.xlsx 
--First line candidate 11
[
    {
        "delegationName": "المدينة_2",
        "data": {
            "Mongi Rahoui": 484,
            "Mohamed Abbou": 3488,
            "Abir Moussi": 3500,
            "Nabil Karoui": 3363,
            "Lotfi Mraihi": 4002,
            "Mehdi Jomaa": 2014,
            "Hamadi Jebali": 36,
            "Hamma Hammami": 278,
            "Moncef Marzouki": 567,
            "Abdelkrim Zbidi": 16576,
            "Mohsen Marzouk": 191,
            "Mohamed Esghaier Nouri": 37,
            "Hechmi Hamdi": 43,
            "Abdelfattah Mourou": 4300,
            "Omar Man

In [24]:
[i for i in range(len(fullMatrix[:,0])) if fullMatrix[i,0] == 1][0]

10

In [3]:
from matplotlib import pyplot as plt
import numpy as np

In [10]:
plotMat = np.array(result)

In [11]:
def initials(name):
    return ('.'.join([i[0] for i in name.split(' ')]))

X = [initials(i) for i in plotMat[1,:,0]]
Y1 = [float(i) for i in plotMat[1,:,1]]
Y2 = [float(i) for i in plotMat[2,:,1]]
Y3 = [float(i) for i in plotMat[3,:,1]]
Y4 = [float(i) for i in plotMat[4,:,1]]
Y5 = [float(i) for i in plotMat[5,:,1]]
fig, ax = plt.subplots()
#fig.set_size_inches((150,150),forward=True)
fig.set_dpi(200)
x = np.arange(len(X))
width = 0.15  # the width of the bars
rects1 = ax.bar(x - 2*width, Y1, width, label='Men')
rects2 = ax.bar(x - width, Y2, width, label='Women')
rects3 = ax.bar(x, Y3, width, label='Women')
rects4 = ax.bar(x + width, Y4, width, label='Women')
rects5 = ax.bar(x + 2*width, Y4, width, label='Women')
ax.set_xticks(x)
ax.set_xticklabels(X,fontsize=5)
#fig.tight_layout()
#fig.set_size_inches(150,20)
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    i=0
    for rect in rects:
        height = rect.get_height()
        label = initials(candidates[i]['nameFR'])
        i=i+1
        
        ax.annotate((label),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    fontsize=7,
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects3)

plt.show()

IndexError: too many indices for array

In [35]:
names = plotMat[1,:,0]

def initials(name):
    return ('.'.join([i[0] for i in name.split(' ')]))
[initials(i) for i in names]

['M.R',
 'M.A',
 'A.M',
 'N.K',
 'L.M',
 'M.J',
 'H.J',
 'H.H',
 'M.M',
 'A.Z',
 'M.M',
 'M.E.N',
 'H.H',
 'A.M',
 'O.M',
 'Y.C',
 'K.S',
 'E.F',
 'S.R',
 'S.E.R',
 'S.A',
 'S.S',
 'N.J',
 'H.B',
 'A.B',
 'S.M']